In [2]:
%load_ext line_profiler
import line_profiler

In [3]:
%load_ext Cython

import Cython
directive_defaults = Cython.Compiler.Options.get_directive_defaults()

In [4]:
# from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True


In [12]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
import numpy as np
cimport numpy as np
import cython

from cython.cimports.libc.math import sin, abs, fabs, M_PI, sqrt

cpdef double[:] Fourier_modes(
    int N_max,
    double L,
):
    cdef:
        int size, n1, n2, n3, m
        double[:] modes
        double kf_E1

    size = (N_max + 1) * (N_max + 2) * (N_max + 3) // 6
    modes = np.empty(size)
    kf_E1 = 2 * M_PI / L

    for n3 in range(0, N_max + 1):
        for n2 in range(0, n3 + 1):
            for n1 in range(0, n2 + 1):
                m = n1 + n2 * (n2 + 1) // 2 + n3 * (n3 + 1) * (n3 + 2) // 6
                modes[m] = kf_E1 * sqrt(n1 * n1 + n2 * n2 + n3 * n3)
    return modes

def this_function_causes_line_profiler_to_print_all_the_way_down_here():
    pass

Content of stderr:
In file included from /Users/deyanmihaylov/.cache/ipython/cython/_cython_magic_6aa625bc0fb81c04f39c9724f866c4fc6c9bb166.c:1187:
In file included from /Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940:
/Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
/Users/deyanmihaylov/.cache/ipython/cython/_cython_magic_6aa625bc0fb81c04f39c9724f866c4fc6c9bb166.c:4608:3

In [8]:
profile = line_profiler.LineProfiler(z)
profile.runcall(z, 0.8)
profile.print_stats()


Timer unit: 1e-09 s



/var/folders/v7/w4c0kx6d5bq1lvxw1rnqy7br0000gp/T/ipykernel_43657/1011706639.py:1: UserWarning: Could not extract a code object for the object <_cython_magic_3d45861b52b7d7a68c0ade9e6adca38d36193e42.CubicSpline object at 0x7fc51bc9b400>
  profile = line_profiler.LineProfiler(z)


In [7]:
import numpy as np
import scipy.interpolate as spip

x = np.linspace(0, 100, 10000)
y = np.sin(x)

z = CubicSpline(x, y)

w = spip.CubicSpline(x, y)


In [9]:
%lprun -f CubicSpline CubicSpline(x, y)


/Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/line_profiler/ipython_extension.py:97: UserWarning: Could not extract a code object for the object <class '_cython_magic_3d45861b52b7d7a68c0ade9e6adca38d36193e42.CubicSpline'>
  profile = LineProfiler(*funcs)


Timer unit: 1e-09 s

In [10]:
%timeit z(0.8)

8.33 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
